In [1]:
import time
import os
import shutil
import sys
import h5py
import tensorflow                             as tf
import numpy                                  as np
import seaborn                                as sns
import pandas                                 as pd
import matplotlib.pyplot                      as plt
import numpy.random                           as random
from sklearn.utils                        import shuffle
from sklearn.model_selection              import train_test_split
from tqdm                                 import tqdm
from os                                   import path
import pysftp                                 as sftp
from matplotlib                 import pyplot as plt 
import random                                 as rnd

print("\n======================================================================================================================================")
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))


 TensorFlow version: 2.6.0
 Eager execution: True


In [2]:
WORKSPACE_PATH      = os.environ['WORKSPACE_PATH']
SurQCTFldr          = WORKSPACE_PATH + '/SurQCT/surqct/'

RatesType           = 'KDiss'
NNRunIdx            = 8

PathToRunFld        = SurQCTFldr + '/../' + RatesType + '/all_temperatures/' 

TTran               = 20000.0

iIdxVec             = np.array([1, 1000, 2000, 3000, 4000, 4581, 5000, 6000], dtype=int)

In [3]:
#===================================================================================================================================
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading  import read_levelsdata, read_diatdata

InputFld = PathToRunFld + '/Run_' + str(NNRunIdx) + '/'
print(InputFld)
sys.path.insert(0, InputFld)
#===================================================================================================================================


[SurQCT]: Loading Modules and Functions ...
/Users/sventuri/WORKSPACE//SurQCT/surqct//../KDiss/all_temperatures//Run_8/


In [4]:
#===================================================================================================================================
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)

Prefix                    = 'Run_'
InputData.NNRunIdx        = NNRunIdx
InputData.PathToRunFld    = InputData.PathToRunFld+'/'+Prefix+str(InputData.NNRunIdx)
InputData.PathToFigFld    = InputData.PathToRunFld+'/'+InputData.PathToFigFld
InputData.PathToParamsFld = InputData.PathToRunFld+'/'+InputData.PathToParamsFld
InputData.PathToDataFld   = InputData.PathToRunFld+'/Data/'                                                               
InputData.PathToParamsFld = InputData.PathToRunFld+'/Params/' 

# InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/N3_NASA.hdf5'
# InputData.Molecules       = ['N2','N2'] 
# InputData.PathToLevelsFile= [InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_LeRoy_nd_ELog.csv',
#                              InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_LeRoy_nd_ELog.csv']
# InputData.PathToDiatFile  = [InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp',
#                              InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp']   
# #InputData.PathToGrouping  = self.WORKSPACE_PATH  + '/Air_Database/Run_0D/database/grouping/O3_UMN/O2/LevelsMap_DPM45.csv'   
# #===================================================================================================================================


[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...


In [5]:
#===================================================================================================================================
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model


InputData.DefineModelIntFlg  = 0
InputData.TrainIntFlg        = 0
InputData.TransferFlg        = False
NN_KDiss                     = model(InputData, InputData.PathToRunFld, None, None)
NN_KDiss.load_params(InputData.PathToParamsFld)
#===================================================================================================================================


[SurQCT]: Loading Final Modules ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['EVib_i', 'ERot_i', 'VMax_i', 'ro_i', 'rMax_i', 'TTran_i']
[SurQCT]:     y =  KDiss
[SurQCT]:   Loading ML Model from Folder: /Users/sventuri/WORKSPACE//SurQCT/surqct//../KDiss/all_temperatures//Run_8/MyModel
[SurQCT]:   Loading ML Model Parameters from File:  /Users/sventuri/WORKSPACE//SurQCT/surqct//../KDiss/all_temperatures//Run_8/Params//ModelCheckpoint/cp-1328.ckpt


In [6]:
#===================================================================================================================================
OtherVar           = InputData.OtherVar
xVarsVec_i         = InputData.xVarsVec_i + ['vqn','jqn']
xVarsVec           = list(set(xVarsVec_i))
print('[SurQCT]:   Reading Variables: ', xVarsVec)

InputData.MultFact = 1.e+8
MinValueTrain      = 1.e-18 * InputData.MultFact
MinValueTest       = 1.e-18 * InputData.MultFact

NMolecules         = len(InputData.PathToLevelsFile)

#InputData.iLevelsVecTest = list(np.array(InputData.iLevelsVecTest) - 1)

[SurQCT]:   Reading Variables:  ['EVib', 'vqn', 'VMax', 'ERot', 'jqn', 'ro', 'rMax']


In [7]:
#===================================================================================================================================
### Reading Levels Info of Initial and Final Molecules
LevelsData = []
DiatData   = []
NLevels    = []
for iMol in range(NMolecules):

    LevelsDataTemp = read_levelsdata(InputData.PathToLevelsFile[iMol], xVarsVec, '')
    LevelsData.append(LevelsDataTemp)

    DiatDataTemp = read_diatdata(InputData.PathToDiatFile[iMol], InputData.Molecules[iMol], np.array([TTran]), np.array([TTran]))
    DiatData.append(DiatDataTemp)

    NLevelsTemp    = LevelsDataTemp.shape[0]
    NLevels.append(NLevelsTemp)

[SurQCT]:   Reading Molecular Levels Data from: /Users/sventuri/WORKSPACE//Air_Database/Run_0D/database/levels/Active_Sampled/O2_AIAA.csv
[SurQCT]:   Reading Molecular Levels Data from: /Users/sventuri/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp
[SurQCT]:   Reading Molecular Levels Data from: /Users/sventuri/WORKSPACE//Air_Database/Run_0D/database/levels/Active_Sampled/O2_AIAA.csv
[SurQCT]:   Reading Molecular Levels Data from: /Users/sventuri/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp


In [8]:
#=======================================================================================================================================
# Reading Excitation Rates Data 
def read_kdissdata(InputData, PathToHDF5File, TTra, TInt, NProcTypes):
    print('[SurQCT]:       Reading HDF5 File from: ' + PathToHDF5File + ' for Excitation Rates at Temperature ' + str(int(TTra)) + 'K')

    HDF5Exist_Flg = path.exists(PathToHDF5File)
    if (HDF5Exist_Flg):
        f = h5py.File(PathToHDF5File, 'a')
    else:
        f = {'key': 'value'}
        
    try:
        TStr  = 'T_' + str(int(TTra)) + '_' + str(int(TInt)) + '/Rates/'
        grp   = f[TStr]

        Data  = grp["Diss"]
        KDiss = Data[...]

        f.close()   

    except:
        
        KDiss = None
        
    return KDiss
#=======================================================================================================================================

In [9]:
KDissAll = read_kdissdata(InputData, InputData.PathToHDF5File, TTran, TTran, 3)

# plt.plot(KDiss[:,0], 'o')
# plt.yscale('log')

[SurQCT]:       Reading HDF5 File from: /Users/sventuri/WORKSPACE//Air_Database/HDF5_Database/O3_UMN.hdf5 for Excitation Rates at Temperature 20000K


In [ ]:
jNLevels              = len(LevelsData[0])    

### FWD Rates
iiIdxVec              = np.arange(jNLevels)

TTranVec              = np.ones((jNLevels))*TTran
TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
TTranDataTemp.index   = iiIdxVec


iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
iLevelsDataTemp.index = iiIdxVec

iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]

KDiss_NN              = np.exp( NN_KDiss.Model.predict(iLevelsData[NN_KDiss.xTrainingVar]) ) / InputData.MultFact
    
    
    
### Plotting

fig = plt.figure(figsize=(12,8))
if (KDissAll is not None):
    plt.scatter(DiatData[1]['EInt'], KDissAll[:,0], s=20, c='k')
plt.scatter(DiatData[1]['EInt'], KDiss_NN,   s=10, c='r')
plt.yscale('log')
plt.ylim([1.e-12, 1.e-8])

(1e-12, 1e-08)

In [ ]:
KDiss = read_kdissdata(InputData, InputData.PathToHAHDF5File, TTran, TTran, 3)

# plt.plot(KDiss[:,0], 'o')
# plt.yscale('log')

In [ ]:
jNLevels              = len(LevelsData[0])    

### FWD Rates
iiIdxVec              = np.arange(jNLevels)

TTranVec              = np.ones((jNLevels))*TTran
TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
TTranDataTemp.index   = iiIdxVec


iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
iLevelsDataTemp.index = iiIdxVec

iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]

KDiss_NN              = np.exp( NN_KDiss.Model.predict(iLevelsData[NN_KDiss.xTrainingVar]) ) / InputData.MultFact
    
    
    
### Plotting

fig = plt.figure(figsize=(12,8))
plt.scatter(DiatData[1]['EInt'], KDiss_NN,   s=10, c='r')
if (KDiss is not None):
    plt.scatter(DiatData[1]['EInt'], KDiss[:,0], s=20, c='k')
plt.yscale('log')
plt.ylim([1.e-16, 1.e-8])

In [ ]:
if (KDiss is not None):
    fig = plt.figure(figsize=(12,8))
    plt.scatter(DiatData[1]['EInt'], KDissAll[:,0], s=20, c='g')
    plt.scatter(DiatData[1]['EInt'], KDiss[:,0], s=20, c='k')
    plt.yscale('log')
    plt.ylim([1.e-16, 1.e-8])